In [ ]:
# !pip install pyspark 
# !pip install kafka-python

In [1]:
from pyspark.sql import SparkSession
import time
from kafka import KafkaProducer
import json

In [2]:
#this is to send the data to kafka simulating a streaming data
#From here untill the cell marked by ** 

#start the spark session + ensure that the configuration package between sprak and kafka is there
spark = SparkSession.builder \
    .appName("CSVToKafka") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0") \
    .getOrCreate()

print("Session creted")

Session creted


In [3]:
#read the data as a spark df 
csv_file_path = "smoke_detection_iot.csv"  # Change this to the path of your CSV file
df = spark.read.option("header", "true").csv(csv_file_path)

In [4]:
#connect to kafka using producer to get ready to send the data
producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [5]:
# topic name 
kafka_topic = 'input'

In [6]:
# last cell in producing data as a stream each second
# going through the datafram row by row to simulate a streaming data then send the data to the kafka topic 
print("STarting to send the data to kafka")
count = 0
for row in df.collect():
    producer.send(kafka_topic, row.asDict())
    producer.flush()
    if count == 10:
        break
    count += 1
print("data sent!")
# Close the Kafka producer
producer.close()

STarting to send the data to kafka
data sent!


In [7]:
spark.stop()

In [ ]:
# # Convert all rows in the dataframe to a list of dictionaries
# data_to_send = [row.asDict() for row in df.collect()]

# # Convert the list of dictionaries to a JSON string
# json_data = json.dumps(data_to_send)

# # Send the JSON string to the Kafka topic
# producer.send(kafka_topic, json_data.encode('utf-8'))
# producer.flush()

# print("Data sent!")

# # Close the Kafka producer
# producer.close()